In [79]:
from sklearn.preprocessing import MinMaxScaler
%run enc_ohe.ipynb

def minMax():
    dataSet = ohe()
    
    # Applying Standard Scaler
    # Num_cols
    cols = ['tenure', 'monthlyCharges', 'totalCharges', 'avgMonthlyPaid']
    scaler = MinMaxScaler()
    dataSet[cols] = scaler.fit_transform(dataSet[cols])
    
    #dataSet.info()
    #dataSet[cols].describe()
    #print(dataSet[cols])

    return dataSet

In [80]:
def kbest(verb):
    from sklearn.feature_selection import SelectKBest, f_classif
    
    dataSet = minMax()
    
    # KBest
    features = dataSet.drop(['churn'], axis=1)
    target = dataSet['churn']
    
    # Testing different Ks
    #k = [6, 7, 8, 9, 10, 12, 14]
    #for qtd in k:
    #    selector = SelectKBest(score_func=f_classif, k=qtd)
    #    featsSelecteds = selector.fit_transform(features, target)
    #    
    #    selectedCols = features.columns[selector.get_support()]
    #    print(f"{qtd} best attributes")
    #    print(list(selectedCols))
    
    k = 9
    selector = SelectKBest(score_func=f_classif, k=k)
    featuresSelected = selector.fit_transform(features, target)
    
    if verb:
        selectedCols = features.columns[selector.get_support()]
        print(f"KBest processing\n")
        print(f"{k} best attributes \n {selectedCols.tolist()}")
        print(f"\n{'*'*120}\n")
    
    return dataSet, target, featuresSelected

In [81]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

def pca(verb, xy=1, comp=1, numComp=5):
    dataSet, target, featuresSelected = kbest(verb)
    
    if verb:
        print(f"PCA plotting\n")
        if xy:
            pca2d(featuresSelected, target)
            print(f"\n{' '*15+'-'*90}\n")
        pcaDatas = pcaComp(featuresSelected, numComp)
        print(f"\n{'*'*120}\n")
    else:
        pcaDatas = pcaComp(featuresSelected, numComp, verb=0)
        

    return pcaDatas, target

def pca2d(featuresSelected, target):
    # PCA setting
    pca = PCA(n_components=2)
    pcaDatas = pca.fit_transform(featuresSelected)
    explained = pca.explained_variance_ratio_
    
    # Verbal explanation
    for i, ev in enumerate(explained, 1):
        print(f"Component {i}: {ev:.2%}")
    print(f'Total: {explained.sum():.2%}')
    
    # 2D chart
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=pcaDatas[:, 0], y=pcaDatas[:, 1], hue=target, palette='Set1', alpha=0.7)
    plt.title('PCA view (2 components)')
    plt.xlabel('Component 1')
    plt.ylabel('Component 2')
    plt.grid(True)
    plt.show()
    

def pcaComp(featuresSelected, numComp, verb=1):
    pca = PCA(n_components=numComp)
    pcaDatas = pca.fit_transform(featuresSelected)
    explainedVar = pca.explained_variance_ratio_
    
    if verb:
        # Verbal explanation
        for i, ev in enumerate(explainedVar, 1):
            print(f"Component {i}: {ev:.2%}")
        print(f"\nVariance by components: {explainedVar.sum():.2%}")
    
        # Variance chart
        plt.figure(figsize=(8, 5))
        sns.barplot(x=[f'PC{i+1}' for i in range(numComp)], y=explainedVar, color='royalblue')
        plt.title('Variance by component')
        plt.ylabel('Variance proportion')
        plt.xlabel('Component')
        plt.ylim(0, 1)
        plt.grid(axis='y')
        plt.show()

    return pcaDatas

In [82]:
def redimensioning(verb=1):
    pcaDatas, target = pca(verb)
    
    # New dataset, using the 5 most impacting attributes
    dsColsName = [f'component{i+1}' for i in range(5)]
    dataSet = pd.DataFrame(pcaDatas, columns=dsColsName)
    dataSet['churn'] = target.values

    if verb:
        print(f"Redimensioning dataSet\n")
        print(dataSet)
        print(f"\n{'*'*120}\n")
    
    return dataSet

In [84]:
def minMaxReduced(verb=1): #verb=1 to print; verb=0 to not
    dataSet = redimensioning(verb)
    return dataSet